# FCCee Analysis Examples
Using the example `higgs/mH-recoil/mumu` from [FCCAnalyses](https://github.com/HEP-FCC/FCCAnalyses)

In [1]:
using EDM4hep
using EDM4hep.RootIO
using EDM4hep.SystemOfUnits
using EDM4hep.Histograms

Precompiling EDM4hep
  ✓ EDM4hep
  1 dependency successfully precompiled in 5 seconds. 225 already precompiled.


## Definition of some analysis functions
These are couple of examples of high-level functions that makes use of `ReconstructedParticle` objects to build resonances and recoils. They make use of standard Julia functions to generate combinations, to sort a vector, and to work with LorentzVectors.  

In [2]:
# re-using convenient existing packages 
using LorentzVectorHEP
using Combinatorics

"""
    resonanceBuilder(rmass::AbstractFloat, legs::AbstractVector{ReconstructedParticle})

Returns a container with the best resonance of 2 by 2 combinatorics of the `legs` container
sorted by closest to the input `rmass` in absolute value.
"""
function resonanceBuilder(rmass::AbstractFloat, legs::AbstractVector{ReconstructedParticle})
    result = ReconstructedParticle[]
    length(legs) < 2 && return result
    for (a,b) in combinations(legs, 2)
        lv = LorentzVector(a.energy, a.momentum...) + LorentzVector(b.energy, b.momentum...)
        rcharge = a.charge + b.charge
        push!(result, ReconstructedParticle(mass=mass(lv), momentum=(lv.x, lv.y, lv.z), charge=rcharge))
    end
    sort!(result, lt =  (a,b) -> abs(rmass-a.mass) < abs(rmass-b.mass))
    return result[1:1]  # take the best one
end;

"""
    recoilBuilder(comenergy::AbstractFloat, legs::AbstractVector{ReconstructedParticle})

    build the recoil from an arbitrary list of input `ReconstructedParticle`s and the center of mass energy.
"""
function recoilBuilder(comenergy::AbstractFloat, in::AbstractVector{ReconstructedParticle})
    result = ReconstructedParticle[]
    isempty(in) && return result
    recoil_lv = LorentzVector(comenergy, 0, 0, 0)
    for p in in
        recoil_lv -= LorentzVector(p.mass, p.momentum...)
    end
    push!(result, ReconstructedParticle(mass=mass(recoil_lv), momentum=(recoil_lv.x, recoil_lv.y, recoil_lv.z)))
    return result
end;

## Defining the Final N-tuple
We create a custom structure with all summary information of each event.

In [18]:
using DataFrames
df = DataFrame(Zcand_m = Float32[], Zcand_recoil_m = Float32[], Zcand_q = Int32[], Zcand_recoil_θ = Float32[]);

## Open the data file to get the events
- It is using a file in EOS with the `root:` protocol
- The obtained `events` is a `LazyTree` created by the [UnROOT.jl](https://github.com/JuliaHEP/UnROOT.jl) package. As the name indicates no event is actually read yet.

In [19]:
f = "root://eospublic.cern.ch//eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/p8_ee_ZZ_ecm240/events_000189367.root"

reader = RootIO.Reader(f);
events = RootIO.get(reader, "events");

## Loop over events and fill the DataFrame

In [20]:
@time for evt in events
    #---get the collection of ReconstructedParticles and Muons
    recps = RootIO.get(reader, evt, "ReconstructedParticles");
    muons = RootIO.get(reader, evt, "Muon#0"; btype=ObjectID{ReconstructedParticle})
    
    sel_muons = filter(x -> pₜ(x) > 10GeV, muons)
    zed_leptonic = resonanceBuilder(91GeV, sel_muons)
    zed_leptonic_recoil = recoilBuilder(240GeV, zed_leptonic)
    if length(zed_leptonic) == 1    #  Filter to have exactly one Z candidate
        Zcand_m        = zed_leptonic[1].mass
        Zcand_recoil_m = zed_leptonic_recoil[1].mass
        Zcand_recoil_θ = zed_leptonic_recoil[1].momentum |> EDM4hep.θ
        Zcand_q        = zed_leptonic[1].charge
        
        if 80GeV <= Zcand_m <= 100GeV
            push!(df, (Zcand_m, Zcand_recoil_m, Zcand_q, Zcand_recoil_θ))
        end
    end
end

 28.983505 seconds (42.87 M allocations: 9.694 GiB, 6.04% gc time, 0.25% compilation time)


## Plot the results

In [21]:
#using Plots
#histogram(df.Zcand_m, title="Resonance mass plot",xlabel="GeV")

In [22]:
#histogram(df.Zcand_recoil_m)

In [23]:
#histogram(df.Zcand_q)

In [24]:
#using CSV
#CSV.write("m_H-recoil.csv", df)

In [25]:
using Parquet2
Parquet2.writefile("m_H-recoil.parquet", df)

✏ Parquet2.FileWriter{IOStream}(m_H-recoil.parquet)